In [76]:
# Loading environmental variables 
from dotenv import load_dotenv
load_dotenv()


True

In [77]:
# Importing for system files
import os
import sys
import csv
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [78]:
# For OpenAI client
from openai import OpenAI

# Acquire OpenAI API Key via this URL 
#    https://platform.openai.com/api-keys
# Save Secret in .env file with 
#    OPENAI_API_KEY=YOUR SECRET KEY

openai_client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [79]:
# Import System Files
dir_path = os.path.dirname(os.path.realpath('__file__'))
sys.path.append(dir_path + '/lib/')  
from process_work import process_work

In [80]:
def calculate_cacao_nibs_price(product_title, sales_price, openai_client):
    # Construct the payload with explicit instructions for valid float response
    payload = f"""
    The product title is '{product_title}', and the sales price is {sales_price}. 
    If the product title refers to cacao nibs and includes the weight in ounces or grams, calculate the cost for 8 ounces.
    Respond with only the price as a float, and nothing else.
    """


    work_process_completion = openai_client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": payload,
            }
        ],
        model="gpt-3.5-turbo",
    )
    
    # Strip the response and ensure it returns a float
    response = work_process_completion.choices[0].message.content.strip()
    
    return response


In [81]:
result = calculate_cacao_nibs_price( "Viva Naturals Organic Cacao Nibs, 2 pounds - Keto Friendly and Vegan", "$25.00", openai_client);
pp.pprint(result)

'$6.25'


In [85]:
def process_cacao_nibs_sales(csv_input_path, csv_output_path, openai_client):
    # Read the input CSV file
    with open(csv_input_path, mode='r', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        products = list(reader)
    
    # Prepare the output data
    output_data = []
    for row in products:
        product_title = row['product_title']
        sales_price = row['pricing']
        
        # Call the OpenAI function to calculate the price
        result = calculate_cacao_nibs_price(product_title, sales_price, openai_client)
        
        # Add the result to the output data
        final_output = {
            'product_title': product_title,
            'pricing': sales_price,
            'calculated_price': result
        }
        output_data.append(final_output)
        
        pp.pprint(final_output)
    
    # Write the output to a new CSV file
    os.makedirs(os.path.dirname(csv_output_path), exist_ok=True)
    with open(csv_output_path, mode='w', newline='') as csvfile:
        fieldnames = ['product_title', 'pricing', 'calculated_price']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        writer.writerows(output_data)



In [86]:
# Example usage
file_key = '20240923_cacao_nibs_sales_prices'
csv_input_path = f"data/{file_key}.csv"
csv_output_path = f"analysis/{file_key}.csv"

process_cacao_nibs_sales(csv_input_path, csv_output_path, openai_client)


{   'calculated_price': '$9.39',
    'pricing': '$14.09',
    'product_title': 'BECCA Ultimate Coverage 24 Hour Foundation, 1 FL. OZ. '
                     'NIB'}
{   'calculated_price': '$6.25',
    'pricing': '$25.00',
    'product_title': 'Viva Naturals Organic Cacao Nibs, 2 Lb Bag (907G) - Keto '
                     'Friendly and Vegan'}
{   'calculated_price': '$196.00',
    'pricing': '$24.50',
    'product_title': 'NIB New Cymbiotika SLEEP Liposomal Sleep Cacao 1.7 Fl Oz'}
{   'calculated_price': '$9.53',
    'pricing': '$5.30',
    'product_title': 'Organic Coconut Bites, with Cacao Nibs & Dark Chocolate, '
                     '5.25 oz (149 g)'}
{   'calculated_price': '9.99',
    'pricing': '$9.99',
    'product_title': '8 ounce organic gourmet cacao nibs from Brazil'}
{   'calculated_price': '$16.98',
    'pricing': '$8.49',
    'product_title': 'Premium Pure Organic Roasted Cocoa Nibs Cacao Great Best '
                     'Quality 2 oz up to 10LB'}
{   'calculated_price